<a href="https://colab.research.google.com/github/bytefirst7/practical_app2/blob/main/prompt_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary.

In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go

from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [35]:
df = pd.read_csv('data/vehicles.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [36]:
df.rename(columns={'region':'city'}, inplace=True)

In [37]:
df.dropna(subset=['year', 'manufacturer', 'model', 'condition', 'cylinders','fuel','odometer','title_status','transmission','VIN','drive','size','type','paint_color'], inplace=True)
df = df.drop('id', axis=1)
df.head(5)

,city,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
126,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,68472.0,clean,automatic,1GCWGAFP8J1309579,rwd,full-size,van,white,al
127,auburn,0,2019.0,chevrolet,express cargo van,like new,6 cylinders,gas,69125.0,clean,automatic,1GCWGAFP4K1214373,rwd,full-size,van,white,al
128,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,66555.0,clean,automatic,1GCWGAFPXJ1337903,rwd,full-size,van,white,al
215,birmingham,4000,2002.0,toyota,echo,excellent,4 cylinders,gas,155000.0,clean,automatic,JTDBT123520243495,fwd,compact,sedan,blue,al
219,birmingham,2500,1995.0,bmw,525i,fair,6 cylinders,gas,110661.0,clean,automatic,WBAHD6322SGK86772,rwd,mid-size,sedan,white,al


### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [38]:
df_encoding = pd.get_dummies(df, columns=['condition'])
df_encoding.head()

,city,price,year,manufacturer,model,cylinders,fuel,odometer,title_status,transmission,...,size,type,paint_color,state,condition_excellent,condition_fair,condition_good,condition_like new,condition_new,condition_salvage
126,auburn,0,2018.0,chevrolet,express cargo van,6 cylinders,gas,68472.0,clean,automatic,...,full-size,van,white,al,False,False,False,True,False,False
127,auburn,0,2019.0,chevrolet,express cargo van,6 cylinders,gas,69125.0,clean,automatic,...,full-size,van,white,al,False,False,False,True,False,False
128,auburn,0,2018.0,chevrolet,express cargo van,6 cylinders,gas,66555.0,clean,automatic,...,full-size,van,white,al,False,False,False,True,False,False
215,birmingham,4000,2002.0,toyota,echo,4 cylinders,gas,155000.0,clean,automatic,...,compact,sedan,blue,al,True,False,False,False,False,False
219,birmingham,2500,1995.0,bmw,525i,6 cylinders,gas,110661.0,clean,automatic,...,mid-size,sedan,white,al,False,True,False,False,False,False


In [39]:
df_encoding = pd.get_dummies(df, columns=['fuel'])
df_encoding.head()

,city,price,year,manufacturer,model,condition,cylinders,odometer,title_status,transmission,...,drive,size,type,paint_color,state,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other
126,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,68472.0,clean,automatic,...,rwd,full-size,van,white,al,False,False,True,False,False
127,auburn,0,2019.0,chevrolet,express cargo van,like new,6 cylinders,69125.0,clean,automatic,...,rwd,full-size,van,white,al,False,False,True,False,False
128,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,66555.0,clean,automatic,...,rwd,full-size,van,white,al,False,False,True,False,False
215,birmingham,4000,2002.0,toyota,echo,excellent,4 cylinders,155000.0,clean,automatic,...,fwd,compact,sedan,blue,al,False,False,True,False,False
219,birmingham,2500,1995.0,bmw,525i,fair,6 cylinders,110661.0,clean,automatic,...,rwd,mid-size,sedan,white,al,False,False,True,False,False


In [40]:
df_encoding = pd.get_dummies(df, columns=['cylinders'])
df_encoding.head()

,city,price,year,manufacturer,model,condition,fuel,odometer,title_status,transmission,...,paint_color,state,cylinders_10 cylinders,cylinders_12 cylinders,cylinders_3 cylinders,cylinders_4 cylinders,cylinders_5 cylinders,cylinders_6 cylinders,cylinders_8 cylinders,cylinders_other
126,auburn,0,2018.0,chevrolet,express cargo van,like new,gas,68472.0,clean,automatic,...,white,al,False,False,False,False,False,True,False,False
127,auburn,0,2019.0,chevrolet,express cargo van,like new,gas,69125.0,clean,automatic,...,white,al,False,False,False,False,False,True,False,False
128,auburn,0,2018.0,chevrolet,express cargo van,like new,gas,66555.0,clean,automatic,...,white,al,False,False,False,False,False,True,False,False
215,birmingham,4000,2002.0,toyota,echo,excellent,gas,155000.0,clean,automatic,...,blue,al,False,False,False,True,False,False,False,False
219,birmingham,2500,1995.0,bmw,525i,fair,gas,110661.0,clean,automatic,...,white,al,False,False,False,False,False,True,False,False


In [41]:
df_encoding = pd.get_dummies(df, columns=[
    'condition',
    'cylinders',
    'transmission',
    'title_status',
    'type',
    'size',
    'drive',
    'fuel'
])

df_encoding.head()

,city,price,year,manufacturer,model,odometer,VIN,paint_color,state,condition_excellent,...,size_mid-size,size_sub-compact,drive_4wd,drive_fwd,drive_rwd,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other
126,auburn,0,2018.0,chevrolet,express cargo van,68472.0,1GCWGAFP8J1309579,white,al,False,...,False,False,False,False,True,False,False,True,False,False
127,auburn,0,2019.0,chevrolet,express cargo van,69125.0,1GCWGAFP4K1214373,white,al,False,...,False,False,False,False,True,False,False,True,False,False
128,auburn,0,2018.0,chevrolet,express cargo van,66555.0,1GCWGAFPXJ1337903,white,al,False,...,False,False,False,False,True,False,False,True,False,False
215,birmingham,4000,2002.0,toyota,echo,155000.0,JTDBT123520243495,blue,al,True,...,False,False,False,True,False,False,False,True,False,False
219,birmingham,2500,1995.0,bmw,525i,110661.0,WBAHD6322SGK86772,white,al,False,...,True,False,False,False,True,False,False,True,False,False


### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`.

In [42]:
df = pd.get_dummies(df, columns=[
    'condition',
    'cylinders',
    'transmission',
    'title_status',
    'type',
    'size',
    'drive',
    'fuel',
    'manufacturer',
    'state',
    'paint_color'
])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34868 entries, 126 to 426836
Columns: 158 entries, city to paint_color_yellow
dtypes: bool(152), float64(2), int64(1), object(3)
memory usage: 6.9+ MB


In [43]:
df.head()

,city,price,year,model,odometer,VIN,condition_excellent,condition_fair,condition_good,condition_like new,...,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
126,auburn,0,2018.0,express cargo van,68472.0,1GCWGAFP8J1309579,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
127,auburn,0,2019.0,express cargo van,69125.0,1GCWGAFP4K1214373,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
128,auburn,0,2018.0,express cargo van,66555.0,1GCWGAFPXJ1337903,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
215,birmingham,4000,2002.0,echo,155000.0,JTDBT123520243495,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
219,birmingham,2500,1995.0,525i,110661.0,WBAHD6322SGK86772,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [44]:
df = df.drop(columns=['city', 'VIN', 'model'])

df.head()

,price,year,odometer,condition_excellent,condition_fair,condition_good,condition_like new,condition_new,condition_salvage,cylinders_10 cylinders,...,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
126,0,2018.0,68472.0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
127,0,2019.0,69125.0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
128,0,2018.0,66555.0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
215,4000,2002.0,155000.0,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
219,2500,1995.0,110661.0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34868 entries, 126 to 426836
Columns: 155 entries, price to paint_color_yellow
dtypes: bool(152), float64(2), int64(1)
memory usage: 6.1 MB


In [46]:
df = df.astype({
    'year': 'int',
    'odometer': 'int',
    'price': 'int'
})

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34868 entries, 126 to 426836
Columns: 155 entries, price to paint_color_yellow
dtypes: bool(152), int64(3)
memory usage: 6.1 MB


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [47]:
# Standard Linear Regression Model

X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred))

coefficients = pd.Series(model.coef_, index=X.columns)
print(coefficients.sort_values(ascending=False))

R² Score: 0.39856889136228835
RMSE: 113601674.23468278
manufacturer_ferrari            69835.053909
manufacturer_aston-martin       39819.164533
manufacturer_tesla              21976.202446
manufacturer_datsun             15110.593821
manufacturer_porsche            10329.966888
                                    ...     
state_ar                        -8029.802188
manufacturer_land rover         -9076.740027
state_ok                        -9208.553731
manufacturer_fiat              -10186.281968
manufacturer_harley-davidson   -13934.716472
Length: 154, dtype: float64


In [48]:
# Ridge Regression Model

X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
ridge_pred = ridge.predict(X_test_scaled)

print("Ridge R²:", r2_score(y_test, ridge_pred))
print("Ridge RMSE:", mean_squared_error(y_test, ridge_pred))

ridge_coefs = pd.Series(ridge.coef_, index=X.columns).sort_values()

Ridge R²: 0.39857134502502034
Ridge RMSE: 113601210.7731319


In [49]:
#Lasso Regression Model
X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
lasso_pred = lasso.predict(X_test_scaled)

print("Lasso R²:", r2_score(y_test, lasso_pred))
print("Lasso RMSE:", mean_squared_error(y_test, lasso_pred))

lasso_coefs = pd.Series(lasso.coef_, index=X.columns).sort_values()

print("Top positive Lasso features:")
print(lasso_coefs[lasso_coefs > 0].tail())

print("\nTop negative Lasso features:")
print(lasso_coefs[lasso_coefs < 0].head())

Lasso R²: 0.39857137448511815
Lasso RMSE: 113601205.20854372
Top positive Lasso features:
drive_4wd               1094.599032
size_full-size          1107.349031
manufacturer_ferrari    1322.353002
year                    3173.093678
fuel_diesel             3453.219473
dtype: float64

Top negative Lasso features:
odometer                -2352.251060
state_ok                -1498.570920
type_sedan              -1368.676323
transmission_other      -1261.774884
cylinders_4 cylinders   -1183.961165
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.533e+11, tolerance: 5.000e+08



### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [50]:
N = 20
top_coeffs = coefficients.reindex(coefficients.abs().sort_values(ascending=False).index).head(N)

fig = px.bar(
    x=top_coeffs.values,
    y=top_coeffs.index,
    orientation='h',
    color=top_coeffs.values,
    color_continuous_scale='RdBu',
    title=f'Top {N} Features Impacting Vehicle Price',
    labels={'x': 'Coefficient Value', 'y': 'Feature'}
)

fig.update_layout(yaxis=dict(autorange="reversed"), showlegend=False)
fig.show()



In [51]:
N = 10
top_coeffs = coefficients.reindex(coefficients.abs().sort_values(ascending=False).index).head(N)

fig = px.bar(
    x=top_coeffs.values,
    y=top_coeffs.index,
    orientation='h',
    color=top_coeffs.values,
    color_continuous_scale='RdBu',
    title=f'Top {N} Features Impacting Vehicle Price',
    labels={'x': 'Coefficient Value', 'y': 'Feature'}
)

fig.update_layout(yaxis=dict(autorange="reversed"), showlegend=False)
fig.show()

In [54]:
N = 20
top_features = lasso_coefs.abs().sort_values(ascending=False).head(N).index

# Prepare data
lasso_vals = lasso_coefs.loc[top_features]
ridge_vals = ridge_coefs.loc[top_features]

# Create interactive bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=lasso_vals.values,
    y=lasso_vals.index,
    name='Lasso Coefficients',
    orientation='h',
    marker_color='crimson'
))

fig.add_trace(go.Bar(
    x=ridge_vals.values,
    y=ridge_vals.index,
    name='Ridge Coefficients',
    orientation='h',
    marker_color='royalblue'
))

# Layout
fig.update_layout(
    title=f"Top {N} Feature Coefficients: Lasso vs. Ridge",
    xaxis_title="Coefficient Value",
    yaxis=dict(autorange="reversed"),
    barmode='group',
    height=600
)

fig.show()



### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.